In [346]:
import pandas as pd

In [347]:
# read the input file 
data=pd.read_excel('Assignment_Timecard.xlsx')
display(data)

,Position ID,Position Status,Time,Time Out,Timecard Hours (as Time),Pay Cycle Start Date,Pay Cycle End Date,Employee Name,File Number
0,WFS000054,Active,NaT,NaT,NaN,NaT,NaT,"SiWgh, PraGhjEM",54
1,WFS000065,Active,2023-09-12 10:08:00,2023-09-12 13:53:00,3:45,2023-09-10,2023-09-23,"REsaXiaWE, XAis",65
2,WFS000065,Active,2023-09-12 14:23:00,2023-09-12 19:02:00,4:39,2023-09-10,2023-09-23,"REsaXiaWE, XAis",65
3,WFS000065,Active,2023-09-13 10:08:00,2023-09-13 14:20:00,4:12,2023-09-10,2023-09-23,"REsaXiaWE, XAis",65
4,WFS000065,Active,2023-09-13 14:50:00,2023-09-13 20:44:00,5:54,2023-09-10,2023-09-23,"REsaXiaWE, XAis",65
...,...,...,...,...,...,...,...,...,...
1479,WFS000589,Active,2023-09-20 09:55:00,2023-09-20 14:30:00,4:35,2023-09-10,2023-09-23,"WgAyeW, RayCEWd",589
1480,WFS000589,Active,2023-09-20 15:00:00,2023-09-20 19:29:41,4:29,2023-09-10,2023-09-23,"WgAyeW, RayCEWd",589
1481,WFS000589,Active,2023-09-21 09:56:00,2023-09-21 14:30:00,4:34,2023-09-10,2023-09-23,"WgAyeW, RayCEWd",589
1482,WFS000589,Active,2023-09-21 15:00:01,2023-09-21 19:16:05,4:16,2023-09-10,2023-09-23,"WgAyeW, RayCEWd",589


In [348]:
# get columns and rows length
# print(data.columns.size)
# print(len(data))

In [349]:
# decription of data
# data.describe()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Position ID               1484 non-null   object        
 1   Position Status           1484 non-null   object        
 2   Time                      1474 non-null   datetime64[ns]
 3   Time Out                  1470 non-null   datetime64[ns]
 4   Timecard Hours (as Time)  1474 non-null   object        
 5   Pay Cycle Start Date      1474 non-null   datetime64[ns]
 6   Pay Cycle End Date        1474 non-null   datetime64[ns]
 7   Employee Name             1484 non-null   object        
 8   File Number               1484 non-null   int64         
dtypes: datetime64[ns](4), int64(1), object(4)
memory usage: 104.5+ KB


In [350]:
# get details about null
data.isna().sum(axis=1)

0       5
1       0
2       0
3       0
4       0
       ..
1479    0
1480    0
1481    0
1482    0
1483    5
Length: 1484, dtype: int64

In [351]:
# Missing values in columns
data.isna().sum()

Position ID                  0
Position Status              0
Time                        10
Time Out                    14
Timecard Hours (as Time)    10
Pay Cycle Start Date        10
Pay Cycle End Date          10
Employee Name                0
File Number                  0
dtype: int64

Since there are missing values in time columns, it will not be a bad idea if we fill the missing values with the previous valid value

In [352]:
data.fillna(axis=0,method='ffill',inplace=True)
# if there are columns which first value is missing, then replace it with the next valid value
data.fillna(method='bfill',inplace=True)
data.isna().sum()

C:\Users\aveng\AppData\Local\Temp\ipykernel_13956\2462040513.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(axis=0,method='ffill',inplace=True)
C:\Users\aveng\AppData\Local\Temp\ipykernel_13956\2462040513.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='bfill',inplace=True)


Position ID                 0
Position Status             0
Time                        0
Time Out                    0
Timecard Hours (as Time)    0
Pay Cycle Start Date        0
Pay Cycle End Date          0
Employee Name               0
File Number                 0
dtype: int64

In [353]:
def convert_to_timedelta(time_str):
    # Split the time string into hours, minutes, and seconds
    # print(map(int,time_str.split(":")))
    # print(time_str.split(':'))
    hours, minutes = map(int,time_str.split(':'))
    # # Create a timedelta object
    return pd.Timedelta(hours=hours, minutes=minutes,seconds=0)

In [354]:
# Timecard column is of object type, to check the shift hours, we need to make it comparable.
data['Timecard Hours (as Time)']=data['Timecard Hours (as Time)'].apply(convert_to_timedelta)

Now our data has no missing values, we can perform our queries to get the results

Query:
Programmatically analyze the file and print in console the name and position of employees 
      a) who has worked for 7 consecutive days.
      b) who have less than 10 hours of time between shifts but greater than 1 hour
      c) Who has worked for more than 14 hours in a single shift

Printing name of employee who meets all the three conditions (worked for 7 consecutive days, less than 10 hours between shifts, and worked more than 14 hours in a single shift)

In [356]:
 # a) Employees who have worked for 7 consecutive days
consecutive_days = data.groupby('Employee Name')['Time'].diff().dt.days == 1
employees_7_consecutive_days = data[consecutive_days].groupby('Employee Name').filter(lambda x: len(x) >= 7)


In [357]:
# b) Employees with less than 10 hours between shifts but greater than 1 hour
time_between_shifts = data.groupby('Employee Name')['Time'].diff().dt.total_seconds() / 3600
employees_less_than_10_hours = data[(time_between_shifts < 10) & (time_between_shifts > 1)]

 # c) Employees who worked for more than 14 hours in a single shift
long_shift_employee=data[data['Timecard Hours (as Time)'] > pd.Timedelta(hours=14)]

In [358]:
# Find employees who meet all three conditions
employees_meeting_all_conditions = set(employees_7_consecutive_days['Employee Name']).intersection(set(employees_less_than_10_hours['Employee Name'])).intersection(set(long_shift_employee['Employee Name']))

Final Output

In [359]:
display("Employee Names Are: ")
display(employees_meeting_all_conditions)

'Employee Names Are: '

set()